In [ ]:
import geopandas as gpd
import os
import pandas as pd
import glob
import datetime

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
fold = r"C:\Users\arich\Desktop\Shape Files GIS-20240311T200548Z-001\Shape Files GIS"
main =  os.walk(fold)

sites = {}
for path, dirnames, filenames in main:
    
    for file in filenames:
        if  file.endswith('.shp') and ("sites" in file.lower()):
            print(file)
            # print('{} {} {}'.format(repr(path), repr(dirnames), repr(file)))
            cur = gpd.read_file(os.path.join(path, file))
            sites[file] = cur
            # df.extend([cur])

In [ ]:
import conda_scripts
import numpy as np

In [ ]:
# Path to the output Excel file
output_file = 'discharge\meas.xlsx'

# Create an Excel writer object
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet_name, gdf in sites.items():
        # Convert GeoDataFrame to DataFrame
        df = pd.DataFrame(gdf.drop(columns='geometry'))
        # Write DataFrame to Excel sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
alls = []
for k in sites.keys():
    cols = 'WPName 	 	Date 	 	LAT 	LON  Discharge	SiteName CreekCode geometry  source'.split()
    dfo = sites[k]
    # dfo.loc[:,'source'] = k
    if 'Discharge' in dfo.columns:
        dfo = dfo.astype({'Discharge':float})
        # print(dfo.head(),'\n\n', '-'*20,'\n')
        cols = [x for x in cols if x in dfo.columns]
        alls.extend([dfo.loc[:,cols]])
    else:
        col = dfo.drop(columns = ['geometry','source']).columns[-1]
        # cols = 'WPName 	SiteName 	Date 	 	LAT 	LON 	CreekCode geometry'.split()
        cols = [x for x in cols if x in dfo.columns]
        cols.extend([col])
        
        alls.extend([dfo.loc[:,cols].rename(columns = {col:'Discharge'})])


    
alls = pd.concat([a.to_crs(2226) for a in alls])
alls = alls.loc[alls.geometry.x<np.inf]

In [ ]:
alls.to_file('discharge\streamQ.shp')